In [67]:
import os
import numpy as np
import pandas as pd
from utils import *
from tensorflow.keras.utils import save_img

In [5]:
B = 0
DATA_DIR = 'LizaAlertDroneDatasetV4_Winter'
INPUT_CROP_SIZE, FEATURE_MAP_SIZE = EFFICIENT_NET_SIZES[B]
BIG_CROP_SIZE = int(INPUT_CROP_SIZE * 1.2)
os.listdir(DATA_DIR)

['__pycache__',
 'ImageSets',
 'Annotations',
 'JPEGImages',
 'crops1024',
 '.ipynb_checkpoints']

In [88]:
def annotations2string(i, annotations, SET, INPUT_CROP_SIZE):
    """Google vision API converter"""
    out = []
    for ann in annotations:
        label = ann.label
        x_min = ann.bbox.xmin / INPUT_CROP_SIZE
        x_max = ann.bbox.xmax / INPUT_CROP_SIZE
        y_min = ann.bbox.ymin / INPUT_CROP_SIZE
        y_max = ann.bbox.ymax / INPUT_CROP_SIZE
        out.append(f'{SET},{DATA_DIR}/{SET}crops/{i}.jpg,{label},{x_min:.3f},{y_min:.3f},,,{x_max:.3f},{y_max:.3f},,\n')
    return out

In [85]:
with open(DATA_DIR+'/ImageSets/Main/train.txt') as f:
    train_i = f.read().split('\n')[:-1]

with open(DATA_DIR+'/ImageSets/Main/test.txt') as f:
    test_i = f.read().split('\n')[:-1]

test_idxs = [int(t) for t in test_i]
train_idxs = [int(t) for t in train_i]

In [89]:
def test_gen():
    for i in test_idxs:
        out = crop_sample(i, INPUT_CROP_SIZE, FEATURE_MAP_SIZE, DATA_DIR)
        for o in out:
            _, x, _, anns = o
            yield x, anns

def train_gen():
    for i in train_idxs:
        out = crop_sample(i, INPUT_CROP_SIZE, FEATURE_MAP_SIZE, DATA_DIR)
        for o in out:
            _, x, _, anns = o
            yield x, anns

test_it = iter(test_gen())
train_it = iter(train_gen())

In [90]:
out = []

SET = 'TEST'
! mkdir {DATA_DIR}/{SET}crops
for i, (x, annotations) in enumerate(test_it):
    save_img(f'{DATA_DIR}/{SET}crops/{i}.jpg', x)
    for s in annotations2string(i, annotations, SET, INPUT_CROP_SIZE):
        out.append(s)

SET = 'TRAIN'
! mkdir {DATA_DIR}/{SET}crops
for i, (x, annotations) in enumerate(train_it):
    save_img(f'{DATA_DIR}/{SET}crops/{i}.jpg', x)
    for s in annotations2string(i, annotations, SET, INPUT_CROP_SIZE):
        out.append(s)

with open(f'{DATA_DIR}/vision.csv', 'w') as f:
    f.writelines(out)